# Extension of NGC Heroin FOU Results 
**Author:** Jesse Marks

We have preliminary results from a very large-scale GWAS for cigarette smoking and alcohol consumption phenotypes. We will search for those SNPs in our NGC meta-analysis results. Supplemental tables S6-S10 are here:
`\rcdcollaboration01.rti.ns\GxG\Analysis\GSCAN\shared MS version 1\`

The phenotypes of interest to us include:

1) **Age of smoking initiation (AI)** - supplemental table 6

2) **Cigarettes per day (CPD)**- supplemental table 7

3) **Smoking cessation (SC)** - supplemental table 8

4) **smoking initiation (SI)** - supplemental table 9

5) **Drinks Per Week (DPW)** - supplemental table 10

We are interested in seeing whether these associations extend over to opioid addiction. The NGC FOU meta-analysis results are located on the share drive at:
`s3://rti-midas-data/studies/ngc/meta/040/processing/fou/aand+adaa+alive+cats+cogend+start+uhs1+uhs2-3+vidus+yale-penn.aa+ea.fou.chr*.maf_gt_0.01.rsq_gt_0.3.with_cohort_data.gz`

## Download Data + Create SNP list
The data are located on AWS S3. Create a list of SNPs for the lookup from the GSCAN Excell sheet. Copy the SNPs from each of the five Excell sheets (supplemental tables) into separate files and then combine them while removing duplicates.

In [ ]:
## local ##

# Create directory structure locally
cd ~/Desktop/Projects/heroin/ngc/gscan_lookup/fou
aws s3 cp s3://rti-midas-data/studies/ngc/meta/040/processing/fou/aand+adaa+alive+cats+cogend+start+uhs1+uhs2-3+vidus+yale-penn.aa+ea.fou.maf_gt_0.03.rsq_gt_0.3.table.gz .

# populate these files with the SNPs from respective Excel sheets
touch age_of_initiation.tsv  cig_per_day.tsv  smoking_cessation.tsv  smoking_initiation.tsv drink_week.tsv
wc -l *tsv
"""
   11 age_of_initiation.tsv
   56 cig_per_day.tsv
  101 drink_week.tsv
   25 smoking_cessation.tsv
  377 smoking_initiation.tsv
  570 total
"""

# combine SNPs into one file, make sure SNPs are not listed twice
head -1 age_of_initiation.tsv > combined_snp_list.tsv
for file in age* cig* smoking* drink*;do
    tail -n +2 $file >> combined_snp_list.tsv
done

# filter so that there are no duplicated SNPs (no header either)
tail -n +2 combined_snp_list.tsv | sort -u > combined_snp_list_filtered.tsv

# convert to 1000g_p3 format
awk '{print $3":"$2":"$4":"$5"\t"$1}' combined_snp_list_filtered.tsv > combined-snp_list-filtered-1000g_p3-chr.tsv

```
wc -l combined-snp_list-filtered-1000g_p3-chr.tsv
557 combined-snp_list-filtered-1000g_p3-chr.tsv

head combined-snp_list-filtered-1000g_p3-chr.tsv
rs12027999:154206358:T:C        1
rs2072659:154548521:C:G 1
rs45444697:155034632:C:G        1
rs10753661:165119792:G:A        1
rs28680958:173848808:G:A        1
rs2901785:174104743:G:A 1
rs34973462:175993820:C:T        1
rs147052174:179783167:G:T       1
rs3820277:18436657:G:T  1
rs35656245:190957480:G:A        1
```

## SNP lookup
I need to create a dictionary of the GSCAN SNPs and then see if each SNP in the meta-analysis is in the dictionary. I think this makes more sense than vice-versa; in particular, creating a dictionary for each SNP in the meta-analysis and then searching to see if the GSCAN SNPs are in the dictionary. This latter strategy would require a large amount of memory to create the Python dictionary. I think the former strategy makes more computational sense.

Also note that there are some SNPs in the meta-analysis which have the format of chr:position:a1:a2 instead of rsid:position:a1:a2. I think the reason is that these SNPs of the former format did not have an associated rsID available. If a GSCAN SNP is not found in the lookup, then we need to output the SNPs that were not found and deal with those later. It might be the case that they we have to convert them from rsid:position:a1:a2 format to chr:position:a1:a2 and then perform the search again with just these SNPs.

### Cross-Ancestry

In [2]:
### Python3 ###
"""
*SNP lookup*

    Make sure the IDs are of the same format for the snp-list
    and the IDs in the meta-analysis results. e.g. 1000g_p3 or rsID only
"""
import gzip

################################################################################
date = "20190114"  # enter today's date
#ancestry = "ea"

#if ancestry=="aa":
#    pop = "afr"
#elif ancestry=="ea":
#    pop = "eur"
#else:
#    pop = "afr+eur"

## dict to hold gscan snps and the number of times they were found.
## we can tell which SNPs did not show up in any of the meta files
gscan_dict =  {}
base_dir = "C:\\Users\\jmarks\\Desktop\\gscan_lookup\\fou"
snp_list = "{}\\combined-snp_list-filtered-1000g_p3-chr.tsv".format(base_dir)

#for chrom in range(1,23):
out_file = "{}\\results\\{}-ngc-meta-fou-aa+ea.maf_gt_0.01.rsq_gt_0.3-gscan-lookup.txt".format(base_dir, date)
results = "{}\\aand+adaa+alive+cats+cogend+start+uhs1+uhs2-3+vidus+yale-penn.aa+ea.fou.maf_gt_0.01.rsq_gt_0.3.table.gz".format(base_dir)
not_found = "{}\\results\\{}-ngc-meta-fou-gscan-snps-not-found".format(base_dir, date)
################################################################################

with gzip.open(results, 'rt') as metF, open(snp_list) as gscanF, open(out_file, "wt") as outF:
    gscan_line = gscanF.readline()
    met_head = metF.readline()
    met_line = metF.readline()
    print(gscan_line)
    print(met_head, met_line)

    outF.write(met_head)

    ## create a dictionary containing the gscan snps
    if len(gscan_dict) == 0:
        while gscan_line:
            key = gscan_line.split()[0]
            gscan_dict[key] = 0
            gscan_line = gscanF.readline()


    while met_line:
        met_id = met_line.split()[0] # the 1000g_p3 ID in the meta-analysis

        if met_id in gscan_dict:
            gscan_dict[met_id] += 1
            outF.write(met_line)
        met_line = metF.readline()
        
# report SNPs not found
with open(not_found, "wt") as notF:
    for key, value in gscan_dict.items():
        if value==0:
            notF.write(key + "\n")

rs12027999:154206358:T:C	1

VARIANT_ID	CHR	POSITION	P	TYPE
 rs699737:117270868:C:G	1	117270868	0.7131	snp



```
~/Desktop/gscan_lookup/fou/results
$ head *
==> 20190114-ngc-meta-fou-aa+ea.maf_gt_0.01.rsq_gt_0.3-gscan-lookup.txt <==
VARIANT_ID      CHR     POSITION        P       TYPE
rs4912332:58815243:C:T  1       58815243        0.8664  snp
rs10914684:33795572:G:A 1       33795572        0.9134  snp
rs34973462:175993820:C:T        1       175993820       0.6957  snp
rs951740:44011737:G:A   1       44011737        0.4434  snp
rs11264100:35591626:A:G 1       35591626        0.3141  snp
rs147052174:179783167:G:T       1       179783167       0.04811 snp
rs3820277:18436657:G:T  1       18436657        0.5824  snp
rs12088813:66407700:A:C 1       66407700        0.4966  snp
rs35656245:190957480:G:A        1       190957480       0.3217  snp

==> 20190114-ngc-meta-fou-gscan-snps-not-found <==
rs184083806:96981736:T:C
rs2145451:29316842:T:C
rs4886550:78243579:A:G
rs12442563:83893243:G:T
rs72836318:44121579:T:C
rs3076896:146283610:G:A
rs12611472:225353649:T:C
rs28813180:158083918:G:A
rs561222871:100260679:C:T
rs11739827:166803321:G:T
```